In [2]:
import os
import pandas as pd
import numpy as np
import numba

In [42]:
df_main=pd.read_csv('data/normalized_filtered_user_listening.csv').drop(['Unnamed: 0'], axis=1)
# df=pd.read_csv('data/user_listening.csv')

In [49]:
df_main.head()

,track_id,user_id,normalized_playcount
0,TRAAHSY128F147BB5C,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1.0
1,TRPGYLT128F428AD02,85c1f87fea955d09b4bec2e36aee110927aedf9a,1.0
2,TRWCEKE128F93191BE,bd4c6e843f00bd476847fb75c47b4fb430a06856,1.0
3,TRDSFKT12903CB510F,4bd88bfb25263a75bbdd467e74018f4ae570e5df,0.5
4,TRRELZC128E078ED67,4bd88bfb25263a75bbdd467e74018f4ae570e5df,1.0


In [54]:
df=df_main[0:10000]
# df.set_index(["user_id", "track_id"], inplace=True)
# # Create a mapping from user IDs and track IDs to numeric indices
# user_id_to_index = {user_id: i for i, user_id in enumerate(df.index.get_level_values(0).unique())}
# track_id_to_index = {track_id: i for i, track_id in enumerate(df.index.get_level_values(1).unique())}

# # Replace the strings with numeric indices
# df['user_id'] = df.index.get_level_values(0).map(user_id_to_index)
# df['track_id'] = df.index.get_level_values(1).map(track_id_to_index)

# Create a mapping from user IDs and track IDs to numeric indices
user_id_to_index = {user_id: i for i, user_id in enumerate(df.user_id.unique())}
track_id_to_index = {track_id: i for i, track_id in enumerate(df.track_id.unique())}

# Replace the strings with numeric indices
df['user'] = df.user_id.map(user_id_to_index)
df['track'] = df.track_id.map(track_id_to_index)

df.set_index(["user", "track"], inplace=True)

C:\Users\metin\AppData\Local\Temp\ipykernel_13608\3780139360.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user'] = df.user_id.map(user_id_to_index)
C:\Users\metin\AppData\Local\Temp\ipykernel_13608\3780139360.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['track'] = df.track_id.map(track_id_to_index)


In [69]:
df_np=df_main
matrix=df.pivot(index='track_id',columns='user_id',values='normalized_playcount')
# matrix=df.pivot(index='track_id',columns='user_id',values='playcount')
arr=np.array(matrix)

In [22]:
non_nan_values = np.logical_not(np.isnan(arr))
non_nan_indices = np.where(non_nan_values)
non_nan_elements = arr[non_nan_indices]
print(non_nan_indices)
print(arr[1][11])
print(non_nan_elements)

(array([   0,    1,    1, ..., 5075, 5076, 5076], dtype=int64), array([1508,   11, 1656, ...,  698,  254, 1628], dtype=int64))
0.5
[1.         0.5        0.4        ... 0.33333333 0.21052632 0.5       ]


In [59]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix


"""Load the user artists file and return a user-artists matrix in csr
    fromat.
    """
# user_artists = pd.read_csv(user_artists_file, sep="\t")
# df.set_index(["user_id", "track_id"], inplace=True)
coo = coo_matrix(
    (
        df.normalized_playcount.astype(float),
        (
            df.index.get_level_values(0),
            df.index.get_level_values(1),
        ),
    )
)
arrcrr=coo.tocsr()

In [61]:
print(arrcrr)

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	0.5
  (3, 4)	1.0
  (3, 5)	0.25
  (4, 6)	0.3333333333333333
  (4, 7)	1.0
  (4, 8)	0.3333333333333333
  (4, 9)	0.6666666666666666
  (5, 10)	0.6666666666666666
  (5, 11)	0.3333333333333333
  (5, 12)	0.6666666666666666
  (5, 13)	0.3333333333333333
  (5, 14)	0.3333333333333333
  (5, 15)	0.6666666666666666
  (5, 16)	0.6666666666666666
  (5, 17)	1.0
  (5, 18)	0.3333333333333333
  (5, 19)	0.3333333333333333
  (5, 20)	0.3333333333333333
  (6, 21)	0.25
  (6, 22)	0.25
  (6, 23)	1.0
  (6, 24)	0.25
  :	:
  (1871, 3729)	0.0625
  (1871, 5064)	1.0
  (1871, 5065)	0.0625
  (1871, 5066)	0.0625
  (1871, 5067)	0.0625
  (1872, 5068)	1.0
  (1872, 5069)	0.25
  (1873, 105)	0.5555555555555556
  (1873, 3812)	1.0
  (1873, 5070)	0.1111111111111111
  (1874, 328)	0.2727272727272727
  (1874, 1580)	0.1818181818181818
  (1874, 1720)	0.7272727272727273
  (1874, 2534)	0.1818181818181818
  (1874, 4586)	0.1818181818181818
  (1874, 5071)	0.1818181818181818
  (1874, 5072)	0.272

In [7]:
print('Unique user count :' , len(df['user_id'].unique()))
print('Unique track count :' , len(df['track_id'].unique()))

Unique user count : 1877
Unique track count : 5077


In [73]:
#filled elements
known_values=list(zip(np.where(~np.isnan(arr))[0],
        np.where(~np.isnan(arr))[1]))

#randomly select 30% as testing set
lucky_draw=set(np.random.choice(range(len(known_values)),
                 size=int(len(known_values)*0.3),
                 p=[1/len(known_values)]*len(known_values)))

unlucky_draw=[i for i in range(len(known_values)) if i not in lucky_draw]
testing_idx=(np.where(~np.isnan(arr))[0][list(lucky_draw)],
np.where(~np.isnan(arr))[1][list(lucky_draw)])
training_idx=(np.where(~np.isnan(arr))[0][list(unlucky_draw)],
np.where(~np.isnan(arr))[1][list(unlucky_draw)])

#train test split
mask=np.ones(arr.shape)
mask[testing_idx]=np.nan
arr_train=np.multiply(arr,mask)

# Convert arr_train to CSR format
arr_csr = csr_matrix(arr_train)

# SPARSE TRIAL

## FUNK SVD

In [86]:
@numba.njit
def funk_svd_epoch_sparse(arr, miu, b_u, b_i, p_u, q_i, alpha, lambda_):
    # initialize
    error = 0

    # only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui = arr[i, u]

            # another way to identify nan
            # r_ui != r_ui
            if np.isnan(r_ui):
                continue

            # compute error
            epsilon_ui = r_ui - miu - b_u[u] - b_i[i] - q_i[i].T @ p_u[u]
            error += epsilon_ui**2

            # update
            b_u[u] += alpha * (epsilon_ui - lambda_ * b_u[u])
            b_i[i] += alpha * (epsilon_ui - lambda_ * b_i[i])
            p_u[u] += alpha * (epsilon_ui * q_i[i] - lambda_ * p_u[u])
            q_i[i] += alpha * (epsilon_ui * p_u[u] - lambda_ * q_i[i])

    return error, b_u, b_i, p_u, q_i

def funk_svd_sparse(arr, miu_init=None, b_u_init=[], b_i_init=[],
                    p_u_init=[], q_i_init=[], num_of_rank=40,
                    alpha=0.005, lambda_=0.02, tau=0.0001,
                    max_iter=20, diagnosis=True):

    # initialize
    stop = False
    counter = 0
    sse = None

    # global mean
    if not miu_init:
        # miu = arr[~np.isnan(arr)].mean()
        miu = arr.mean()
    else:
        miu = miu_init

    # user baseline
    if len(b_u_init) == 0:
        b_u = np.zeros(arr.shape[1])
    else:
        b_u = b_u_init

    # item baseline
    if len(b_i_init) == 0:
        b_i = np.zeros(arr.shape[0])
    else:
        b_i = b_i_init

    # user latent factors
    if len(p_u_init) == 0:
        p_u = np.zeros((arr.shape[1], num_of_rank))
        p_u.fill(0.1)
    else:
        p_u = p_u_init

    # item latent factors
    if len(q_i_init) == 0:
        q_i = np.zeros((arr.shape[0], num_of_rank))
        q_i.fill(0.1)
    else:
        q_i = q_i_init

    # gradient descent
    while not stop:

        error, b_u, b_i, p_u, q_i = funk_svd_epoch_sparse(arr, miu,
                                                          b_u, b_i,
                                                          p_u, q_i,
                                                          alpha, lambda_)

        counter += 1

        # maximum number of epoch
        if counter >= max_iter:
            stop = True
            if diagnosis:
                print('Not converged.',
                      'Consider increasing the number of iterations or tolerance')

        # use sum of squared error to determine if converged
        sse_prev = sse
        sse = error
        if sse_prev and abs(sse / sse_prev - 1) <= tau:
            stop = True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u, b_i, p_u, q_i


In [80]:
#use numba to dramatically boost the speed of linear algebra
#this will be a lot faster than surprise library
@numba.njit
def funk_svd_epoch(arr,miu,b_u,b_i,p_u,q_i,alpha,lambda_):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue

            #compute error
            epsilon_ui=r_ui-miu-b_u[u]-b_i[i]-q_i[i].T@p_u[u]
            error+=epsilon_ui**2

            #update
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            p_u[u]+=alpha*(epsilon_ui*q_i[i]-lambda_*p_u[u])
            q_i[i]+=alpha*(epsilon_ui*p_u[u]-lambda_*q_i[i])
    
    return error,b_u,b_i,p_u,q_i

#svd inspired latent factor model by simon funk
def funk_svd(arr,miu_init=None,b_u_init=[],b_i_init=[],
             p_u_init=[],q_i_init=[],num_of_rank=40,
             alpha=0.005,lambda_=0.02,tau=0.0001,
             max_iter=20,diagnosis=True
             ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #user latent factors
    if len(p_u_init)==0:
        p_u=np.zeros((arr.shape[1],num_of_rank))
        p_u.fill(0.1)
    else:
        p_u=p_u_init
    
    #item latent factors
    if len(q_i_init)==0:
        q_i=np.zeros((arr.shape[0],num_of_rank))
        q_i.fill(0.1)
    else:
        q_i=q_i_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,p_u,q_i=funk_svd_epoch(arr,miu,
                                             b_u,b_i,
                                             p_u,q_i,
                                             alpha,lambda_)

        counter+=1

        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,p_u,q_i

In [87]:
#initialize
num_of_latent_factors=40
max_num_of_epoch=2500
learning_rate=0.01
lagrange_multiplier=0.02
# tolerance=0.005
tolerance=0.01

#funk svd
b_u,b_i,p_u,q_i=funk_svd_sparse(arr_csr,num_of_rank=num_of_latent_factors,
         alpha=learning_rate,
         lambda_=lagrange_multiplier,tau=tolerance,
         max_iter=max_num_of_epoch)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at C:\Users\metin\AppData\Local\Temp\ipykernel_13608\3771288649.py (1)[0m
[1m
File "..\..\AppData\Local\Temp\ipykernel_13608\3771288649.py", line 1:[0m
[1m<source missing, REPL/exec in use?>[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'scipy.sparse._csr.csr_matrix'>[0m


In [12]:
arr_train.shape

(14963, 18657)

In [14]:
#compute global mean
miu=arr_train[~np.isnan(arr_train)].mean()

#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),arr_train.shape[1],axis=1)+q_i@p_u.T

In [21]:
output[:,4]

array([0.61196735, 0.42546818, 0.48176674, ..., 0.62387674, 0.59229196,
       0.38357858])

In [23]:
test=12452
n=0
k=10
sorted_indices=np.argsort(output[:,test])[::-1]
top_10_indices = sorted_indices[n:k]
print(np.sort(output[:,test])[::-1][n:k])
print(top_10_indices)

[0.91359751 0.89740565 0.88965822 0.88705654 0.87544146 0.85904533
 0.85328714 0.8530713  0.851575   0.84638962]
[11887  3354 13271 11941 11987  8939  7887  5379  9177  4515]


In [34]:
output[11887,test]

0.9135975074005096

In [49]:
wth=q_i@p_u.T

In [53]:
# b_i[11887]
j=11887
wtf=np.repeat(b_u.reshape(1,-1),arr_train.shape[0],axis=0)
why=np.repeat(b_i.reshape(-1,1),arr_train.shape[1],axis=1)
print(miu)
print(wtf[j,test])
print(why[j,test])
print(wth[j,test])
print(miu+wtf[j,test]+why[j,test]+wth[j,test])

0.4642951579043497
-0.1626279594616031
0.22913927002944032
0.3827910389283227
0.9135975074005096


In [45]:
wth.shape

(14963, 18657)

In [ ]:
#use mse as benchmark for comparison
mse_funk_svd=np.square((
    output-arr)[testing_idx]).sum()/len(arr[testing_idx])
print('Funk SVD Mean Squared Error:',round(mse_funk_svd,3))

Funk SVD Mean Squared Error: 0.104


## Pearson Correlation with baseline

In [8]:
#obtain pearson correlation with baseline
#if b_u is substituted with arr.mean(axis=1) and arr is a complete matrix
#the function will be equivalent to np.corrcoef(arr.T)
#except the diagonal line would be filled with zeros
@numba.njit
def get_pearson_corr_baseline_matrix(arr,b_u):

    similarity_matrix=np.zeros((arr.shape[1],arr.shape[1]))
    for u in range(arr.shape[1]):
        for v in range(u+1,arr.shape[1]):
            
            #self correlation distorts knn selection
            if u==v:
                continue

            #find items rated by both user u and v
            arr_sub1=arr[:,u]
            arr_sub2=arr[:,v]
            set_u=np.where(~np.isnan(arr_sub1))
            set_v=np.where(~np.isnan(arr_sub2))
            I_uv=set(set_u[0]).intersection(set(set_v[0]))
          
            #avoid the case where two users havent rated any items in common
            if len(I_uv)>0:
                
                #extract ratings of inner join items from user u and v
                arr_u=arr_sub1[np.array(list(I_uv))]
                arr_v=arr_sub2[np.array(list(I_uv))]
                numerator=(arr_u-b_u[u]).T@(arr_v-b_u[v])
                denominator=np.sqrt(
                    (arr_u-b_u[u]).T@(arr_u-b_u[u]))*np.sqrt(
                    (arr_v-b_u[v]).T@(arr_v-b_u[v]))
                pearson_baseline=numerator/denominator
            else:
                pearson_baseline=0

            #symmetric matrix
            similarity_matrix[u,v]=pearson_baseline
            similarity_matrix[v,u]=pearson_baseline
            
    return similarity_matrix

In [16]:
#compute similarity matrix
similarity_matrix=get_pearson_corr_baseline_matrix(arr,b_u)